#### All functions that scrapy labels infomation from etherscan.io are aggregated in this .py file

In [1]:
#----------------------------------interact with mongodb--------------------------------------#

from pymongo import MongoClient
from pymongo import errors

def mongoClient(uri, db_name, collection_name):
    client = MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]
    return collection

erc20 = mongoClient('localhost:27017', 'eth_warning', 'token_address')

In [2]:
#------------------------------------create logger-------------------------------------------#
import logging

logger = logging.getLogger('logger')
logger.setLevel(logging.DEBUG)

fh = logging.FileHandler('./update_erc20_address.log')
fh.setLevel(logging.DEBUG)

# define the output format of handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

logger.addHandler(fh)

In [3]:
#---------------------scrapy all ERC20 tokens address from etherscan.io-------------------------#
import requests
from bs4 import BeautifulSoup
import re

templs = []
cnt = 0
for page in range(1, 19):
    print (page)
    url = 'https://etherscan.io/tokens?p=' +str(page)  
    
    try:
        html = requests.get(url)
    except Exception as e:
        try:
            html = requests.get(url)
        except Exception as e:
            try:
                html = requests.get(url)
            except Exception as e:
                print(e)
                continue
    
    soup = BeautifulSoup(html.text, features = 'lxml')
#    print(soup)
    spoon = soup.tbody.find_all('tr')
    
    for tr in spoon:
        pill = tr.find_all('td')
        addr_info = []
        for td in pill:
            try:
                token = td.find('a', href=True)
                addr_info = [token['href'].split('/')[2].lower()]
                break
            except:
                continue
        if addr_info == []:
            logger.info('Not Found!')
            logger.info([ele.get_text() for ele in pill])
        else:
            addr_info = re.split('\(.*\)', [ele.get_text() for ele in pill][2]) + addr_info 
            if len(addr_info) == 3:
                try:
                    erc20.insert_one({'label':'ERC20', 'name':addr_info[0], 'description':addr_info[1], 'address':addr_info[2]})
                    cnt = cnt+1
                except Exception as e:
                    logger.error(e)
                    logger.info(addr_info)
            else:
                try:
                    depict = ''.join(addr_info[1:-1])
                    erc20.insert_one({'label':'ERC20', 'name':addr_info[0], 'description':depict, 'address':addr_info[2]})
                    cnt += 1
                except:
                    logger.error(addr_info)
            templs.append(addr_info)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [4]:
#---------------------scrapy all ERC721 tokens address from etherscan.io-------------------------#

erc721 = mongoClient('localhost:27017', 'eth_warning', 'token_address')

templs = []
for page in range(1, 22):
    print (page)
    url = 'https://etherscan.io/tokens-nft?p=' +str(page)  
    
    try:
        html = requests.get(url)
    except Exception as e:
        try:
            html = requests.get(url)
        except Exception as e:
            try:
                html = requests.get(url)
            except Exception as e:
                print(e)
                continue
    
    soup = BeautifulSoup(html.text, features = 'lxml')
    spoon = soup.tbody.find_all('tr')
    
    for tr in spoon:
        tds = tr.find_all('td')
        for td in tds:
            try:
                addr = td.find('a', href=True)
                addr = addr['href'].split('/')[2].lower()
                break
            except Exception as e:
                logger.info('Address info Not Found!')
              
        addr_info = [ele.get_text() for ele in tds][2].split('..')[1]
#         print(addr_info)
        addr_info = re.split('\(.*\)', addr_info)
        
        if len(addr_info) == 1:
            templs.append(addr_info + [addr])
            try:
                erc721.insert_one({'label':'erc721', 'name':addr_info[0], 'address':addr})
            except Exception as e:
                logger.error(e)
                
        if len(addr_info) == 2:
            templs.append(addr_info + [addr])
            try:
                erc721.insert_one({'label':'erc721', 'name':addr_info[0],'depict':addr_info[1],'address':addr})
            except Exception as e:
                logger.error(e)
                
        if len(addr_info) > 2:
            depict = ''.join(addr_info[1:])
            templs.append([addr_info[0], depict, addr])
            try:
                erc721.insert_one({'label':'erc721', 'name':addr_info[0],'depict':depict, 'address':addr})
            except Exception as e:
                logger.error(e)
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
